In [2]:
# pip install sentencepiece

In [3]:
import jsonlines
import re

with jsonlines.open('mbpp.jsonl') as reader:
    for obj in reader:
        # Assuming the Python code is in a field named 'code'
        code = obj.get('code', '')
        
        # Find all import statements
        import_statements = re.findall(r'(?:from \S+ import \S+|import \S+)', code)
        # remove commas
        import_statements = [statement.replace(',', '') for statement in import_statements]
        
        # Execute the import statements
        for statement in import_statements:
            try:
                exec(statement)
            except ImportError as e:
                print(f"Failed to execute: {statement}")

Below we use 7b param code llama 2 specialized for Python

In [4]:
# import sentencepiece
# from transformers import LlamaForCausalLM, CodeLlamaTokenizer

# tokenizer = CodeLlamaTokenizer.from_pretrained("codellama/CodeLlama-7b-hf")
# model = LlamaForCausalLM.from_pretrained("codellama/CodeLlama-7b-hf")

In [5]:
# PROMPT = '''Write a function to find the length of the shortest string that has both str1 and str2 as subsequences.
# def super_seq(X, Y, m, n):
# <FILL_ME>
# return result
# '''
# input_ids = tokenizer(PROMPT, return_tensors="pt")["input_ids"]
# generated_ids = model.generate(input_ids, max_new_tokens=128)

# filling = tokenizer.batch_decode(generated_ids[:, input_ids.shape[1]:], skip_special_tokens = True)[0]
# print(PROMPT.replace("<FILL_ME>", filling))

In [6]:
def find_def_until_colon(input_string):
    def_index = input_string.find('def')

    if def_index != -1:
        colon_index = input_string.find(':', def_index)

        if colon_index != -1:
            return input_string[def_index:colon_index + 1]

    return None

In [7]:
def everything_after_def(input_string):
    def_index = input_string.find('def')

    if def_index != -1:
        double_newline_index = input_string.find('\n\n', def_index)
        if double_newline_index != -1:
            result = input_string[def_index:double_newline_index]
            return result
    return None

In [8]:
# import re

# def extract_args_with_function_name(declaration):
#     pattern = r'def\s+(\w+)\s*\((.*?)\)'

#     match = re.search(pattern, declaration)

#     if match:
#         function_name = match.group(1)
#         arguments = match.group(2).split(', ')
#     else:
#         function_name = ""
#         arguments = []

#     if len(arguments) == 0:
#         return function_name, "that takes no arguments"
#     elif len(arguments) == 1:
#         return function_name, f"that takes {arguments[0]} as an argument"
#     else:
#         args_except_last = ", ".join(arguments[:-1])
#         last_arg = arguments[-1]
#         return function_name, f"that takes {args_except_last} and {last_arg} as arguments"

In [30]:
def proper_format(prompt_in, code_in):
    result = ""  # Initialize the result variable
    function_index_to = prompt_in.find("to")
    function_index_that = prompt_in.find("that")
    if function_index_to != -1:
        result = prompt_in[function_index_that + len("to"):].strip()
    elif function_index_that != -1:
        result = prompt_in[function_index_that + len("that"):].strip()

    return result + " \n" + find_def_until_colon(code_in)

In [31]:
# def getCodeFormat(prompt_in, code_in):
#     prompt = proper_format(prompt_in, code_in)

#     input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"]
#     attention_mask = torch.ones_like(input_ids)
#     pad_token_id = tokenizer.eos_token_id

#     generated_ids = model.generate(
#         input_ids,
#         attention_mask=attention_mask,
#         pad_token_id=pad_token_id,
#         max_new_tokens=128,
#         num_return_sequences=1
#     )

#     generated_code = tokenizer.batch_decode(
#         generated_ids[:, input_ids.shape[1]:],
#         skip_special_tokens=True
#     )[0]

#     # Assuming generated_code directly contains the code without placeholders
#     return generated_code


# print(getCodeFormat("Write a python function to find the length of the shortest word.", "def len_log(list1): min=len(list1[0]) for i in list1: if len(i)<min: min=len(i) return min"))

In [32]:
# def getCodeFormat(prompt_in, code_in):
#     PROMPT = proper_format(prompt_in, code_in)
#     input_ids = tokenizer(PROMPT, return_tensors="pt")["input_ids"]

#     attention_mask = torch.ones_like(input_ids)  # Set attention mask to ones (all tokens are attended to)
#     pad_token_id = tokenizer.eos_token_id  # Get the pad token ID

#     generated_ids = model.generate(input_ids, attention_mask=attention_mask, pad_token_id=pad_token_id, max_new_tokens=128, num_return_sequences=1)
#     filling = tokenizer.batch_decode(generated_ids[:, input_ids.shape[1]:], skip_special_tokens = True)[0]
#     print("hello" + filling)
#     responseText = PROMPT.replace("<FILL_ME>", filling)

#     return filling

# print(getCodeFormat("Write a python function to find the length of the shortest word.", "def len_log(list1): min=len(list1[0]) for i in list1: if len(i)<min: min=len(i) return min"))

In [24]:
pip install git+https://github.com/huggingface/transformers.git@main accelerate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


DEPRECATION: Loading egg at /Users/henrybae/miniconda3/envs/transformers/lib/python3.11/site-packages/huggingface_hub-0.19.4-py3.8.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
  Cloning https://github.com/huggingface/transformers.git (to revision main) to /private/var/folders/0h/2pkv9xxd45lfrx3mx45bxhxw0000gn/T/pip-req-build-ob51b2vi
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /private/var/folders/0h/2pkv9xxd45lfrx3mx45bxhxw0000gn/T/pip-req-build-ob51b2vi
  Resolved https://github.com/huggingface/transformers.git to commit 35551f9a0f66a22de4971b4a51b3c172d3b87f95
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.


In [25]:
from transformers import AutoTokenizer
import transformers
import torch

model = "codellama/CodeLlama-7b-Python-hf"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [26]:
def getCodeFormat(prompt_input, code_input):
  sequences = pipeline(
      proper_format(prompt_input, code_input),
      do_sample=True,
      top_k=10,
      temperature=0.1,
      top_p=0.95,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
      max_length=200,
  )
  return everything_after_def(sequences[0]['generated_text'])

# print(getCodeFormat("Write a python function to find the length of the shortest word.", "def len_log(list1): min=len(list1[0]) for i in list1: if len(i)<min: min=len(i) return min"))

In [27]:
import locale
locale.getpreferredencoding = lambda: 'UTF-8'

In [28]:
pip install jsonlines

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


DEPRECATION: Loading egg at /Users/henrybae/miniconda3/envs/transformers/lib/python3.11/site-packages/huggingface_hub-0.19.4-py3.8.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
Note: you may need to restart the kernel to use updated packages.


In [38]:
from contextlib import contextmanager

class TimeoutException(Exception): pass

@contextmanager
def time_limit(seconds):
    def signal_handler(signum, frame):
        raise TimeoutException("Timed out!")
    signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(seconds)
    try:
        yield
    finally:
        signal.alarm(0)

In [39]:
import jsonlines
import torch
import numpy as np
from contextlib import contextmanager
import signal


max_loop = 9999
i = 0

with jsonlines.open('mbpp.jsonl') as reader:
# Loop through each example
    for count, obj in enumerate(reader):
        with jsonlines.open('mbpp_label_llama.jsonl') as reader:
              i = np.sum(np.fromiter([1 for _ in reader], dtype=np.int32))
        if count < i:
                continue
        print("Running Example: " + str(i))
        prompt = obj['text']
        code = obj['code']

        print(find_def_until_colon(code))

        responseText = getCodeFormat(prompt, code)

        print(responseText)

        try:
          exec(responseText)
        except Exception as E:
          print("Code Didn't Run")
          print("Error" + str(E))
          obj['method2_llama_output'] = responseText
          obj['method2_llama_success'] = False
          with jsonlines.open('mbpp_label_llama.jsonl', mode='a') as writer:
                writer.write(obj)
          continue

        # Assertion Code:
        assertion_code = obj['test_list']

        # Execute and print if the code passes the assertions
        success = True
        # Assertion Code:
        try:
          with time_limit(20):
            for assertion in assertion_code:
              print(assertion)
              try:
                exec(assertion)
              except Exception as E:
                # Continue to the next example and print the error
                print("Code Failed Assertions" + str(E))
                success = False
                break
        except TimeoutException:
          print("Assertion timed out. Fail.")
          success = False

        # ...
            
        # Store output and success in the object
        obj['method2_llama_output'] = responseText
        obj['method2_llama_success'] = success


        # Store the results in jsonl file
        with jsonlines.open('mbpp_label_llama.jsonl', mode='a') as writer:
                writer.write(obj)

        # terminate loop for testing
        i += 1
        if i == max_loop:
            break

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Running Example: 245
def babylonian_squareroot(number):
def babylonian_squareroot(number):
    """
    Calculates the square root of a number using the Babylonian method.
    """
    if number < 0:
        raise ValueError("Cannot calculate square root of negative number.")
    elif number == 0:
        return 0
    else:
        guess = number
        i = 0
        while guess*guess!= number:
            i += 1
            guess = (guess + number / guess) / 2.0
        return guess
assert babylonian_squareroot(10)==3.162277660168379


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Code Failed AssertionsTimed out!
Running Example: 246
def lps(str):


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

def mul_even_odd(list1):
    even_num = 1
    odd_num = 1
    for i in list1:
        if i % 2 == 0:
            even_num *= i
        else:
            odd_num *= i
    return even_num * odd_num
assert mul_even_odd([1,3,5,7,4,1,6,8])==4
Code Failed Assertions
Running Example: 784
def tuple_str_int(test_str):


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


def tuple_str_int(test_str):
    test_str = test_str.replace('(', '').replace(')', '').replace(' ', '').replace('[', '').replace(']', '')
    test_str = test_str.split(',')
    test_int = []
    for i in test_str:
        test_int.append(int(i))
    return tuple(test_int)
assert tuple_str_int("(7, 8, 9)") == (7, 8, 9)
assert tuple_str_int("(1, 2, 3)") == (1, 2, 3)
assert tuple_str_int("(4, 5, 6)") == (4, 5, 6)
Running Example: 785
def right_insertion(a, x):


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


def right_insertion(a, x):
    i = 0
    while i < len(a) and a[i] < x:
        i += 1
    return i
assert right_insertion([1,2,4,5],6)==4
assert right_insertion([1,2,4,5],3)==2
assert right_insertion([1,2,4,5],7)==4
Running Example: 786
def text_match_three(text):
